In [1]:
import lifesim
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm
import seaborn as sns
import matplotlib
from matplotlib.lines import Line2D

### Import the catalog in Lay configuration with paper I occurrence statistics

In [2]:
bus_0 = lifesim.Bus()
bus_0.build_from_config(filename='/Users/fdannert/Documents/projects/LIFEsim/working/noise_paper_experiment/server_runs/S01_base_SAG13_April23.yaml')

bus_0.data.import_catalog(input_path='/Users/fdannert/Documents/projects/LIFEsim/working/noise_paper_experiment/server_runs/S01_base_SAG13_April23_catalog.hdf5',
                        noise_catalog=True)

ana = lifesim.SampleAnalysisModule(name='ana')
bus_0.add_module(ana)

ana.get_fundamental_snr()

opt_0 = lifesim.Optimizer(name='opt_0')
bus_0.add_module(opt_0)
ahgs_0 = lifesim.AhgsModule(name='ahgs_0')
bus_0.add_module(ahgs_0)

bus_0.connect(('opt_0', 'ahgs_0'))
opt_0.ahgs()

Importing Catalog...
Beginning Import...
Import completed (Time: 1.072387933731079), changing string object types...
[Done] (Time: 0.06001400947570801)
Importing Noise Catalog...


FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = '/Users/fdannert/Documents/projects/LIFEsim/working/noise_paper_experiment/server_runs/S01_base_SAG13_April23_noise.nc', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
bus_1 = lifesim.Bus()
bus_1.build_from_config(filename='/home/ipa/quanz/user_accounts/fdannert/noise_paper_23/70_results/S01_Lay/'
                        'S01_base_SAG13_April23.yaml')

bus_1.data.import_catalog(input_path='/home/ipa/quanz/user_accounts/fdannert/noise_paper_23/70_results/S01_Lay/'
                                   'S01_base_SAG13_April23_catalog.hdf5',
                        noise_catalog=True)

bus_1.data.options.set_manual(instrumental_opt=True)
opt_1 = lifesim.Optimizer(name='opt_1')
bus_1.add_module(opt_1)
ahgs_1 = lifesim.AhgsModule(name='ahgs_1')
bus_1.add_module(ahgs_1)

bus_1.connect(('opt_1', 'ahgs_1'))
opt_1.ahgs()

### Create the star catalog

In [ ]:
np.sum(bus_0.data.catalog.detected[bus_0.data.catalog.nstar == 5])

In [ ]:
bus_0.data.catalog.detected[bus_0.data.catalog.nstar == 5].shape[0]

In [ ]:
star_catalog = {}
n_universe = np.unique(bus_0.data.catalog.nuniverse).shape[0]

for sn in tqdm(np.unique(bus_0.data.catalog.nstar)):
    star_ex = bus_0.data.catalog[bus_0.data.catalog.nstar == sn]
    star_ex_inst = bus_1.data.catalog[bus_1.data.catalog.nstar == sn]
    star = star_ex.iloc[0]
    
    star_catalog[star.name_s] = {'radius_s': star.radius_s,
                                 'mass_s' : star.mass_s, 
                                 'temp_s': star.temp_s, 
                                 'distance_s': star.distance_s,
                                 'ra': star.ra, 
                                 'dec': star.dec,
                                 'stype': star.stype,
                                 'l_sun': star.l_sun,
                                }
    
    star_catalog[star.name_s]['no_detections_base'] = np.sum(star_ex.detected)/n_universe
    star_catalog[star.name_s]['no_detections_inst'] = np.sum(star_ex_inst.detected)/n_universe
    
    star_catalog[star.name_s]['completeness_base'] = np.sum(star_ex.detected)/star_ex.detected.shape[0]
    star_catalog[star.name_s]['completeness_inst'] = np.sum(star_ex_inst.detected)/star_ex_inst.detected.shape[0]
    
    #detections = []
    #completeness = []
    #for nu in star_ex.nuniverse:
    #    detections.append(np.sum(star_ex.detected[star_ex.nuniverse == nu]))
    #    completeness.append(np.sum(star_ex.detected[star_ex.nuniverse == nu])
    #                       / star_ex.detected[star_ex.nuniverse == nu].shape[0])
    #detections = np.array(detections)
    #completeness = np.array(completeness)
    
    '''
    fig, ax = plt.subplots(ncols=2)
    ax[0].hist(detections)
    ax[0].set_title('detections')
    ax[1].hist(completeness)
    ax[1].set_title('completeness')
    plt.show()
    '''
    
    #star_catalog[star.name_s]['no_detections_base_unc'] = np.std(detections)
    #star_catalog[star.name_s]['completeness_base_unc'] = np.std(completeness)
    
star_catalog = pd.DataFrame(star_catalog).T

In [ ]:
star_catalog['d_completeness'] = - star_catalog.completeness_base + star_catalog.completeness_inst

In [ ]:
def shiftedColorMap(cmap, start=0, midpoint=0.5, stop=1.0, name='shiftedcmap'):
    '''
    Function to offset the "center" of a colormap. Useful for
    data with a negative min and positive max and you want the
    middle of the colormap's dynamic range to be at zero.

    Input
    -----
      cmap : The matplotlib colormap to be altered
      start : Offset from lowest point in the colormap's range.
          Defaults to 0.0 (no lower offset). Should be between
          0.0 and `midpoint`.
      midpoint : The new center of the colormap. Defaults to 
          0.5 (no shift). Should be between 0.0 and 1.0. In
          general, this should be  1 - vmax / (vmax + abs(vmin))
          For example if your data range from -15.0 to +5.0 and
          you want the center of the colormap at 0.0, `midpoint`
          should be set to  1 - 5/(5 + 15)) or 0.75
      stop : Offset from highest point in the colormap's range.
          Defaults to 1.0 (no upper offset). Should be between
          `midpoint` and 1.0.
    '''
    cdict = {
        'red': [],
        'green': [],
        'blue': [],
        'alpha': []
    }

    # regular index to compute the colors
    reg_index = np.linspace(start, stop, 257)

    # shifted index to match the data
    shift_index = np.hstack([
        np.linspace(0.0, midpoint, 128, endpoint=False), 
        np.linspace(midpoint, 1.0, 129, endpoint=True)
    ])

    for ri, si in zip(reg_index, shift_index):
        r, g, b, a = cmap(ri)

        cdict['red'].append((si, r, r))
        cdict['green'].append((si, g, g))
        cdict['blue'].append((si, b, b))
        cdict['alpha'].append((si, a, a))

    newcmap = matplotlib.colors.LinearSegmentedColormap(name, cdict)
    plt.register_cmap(cmap=newcmap)

    return newcmap

In [ ]:
customPalette = sns.diverging_palette(42, 225, l=75, as_cmap=True, center='dark', sep=1)
#141

customPalette = shiftedColorMap(customPalette, midpoint=0.8, name='shifted10')

In [ ]:
mask = np.logical_and.reduce((star_catalog.d_completeness != 0,
                             np.invert(star_catalog.not_observed),
                             np.invert(star_catalog.new_observed)))


In [ ]:
color = ['#ECA86E', '#6BC2E4']

small_size = 7
medium_size = 9
large_size = 10

plt.rc('font', size=small_size)          # controls default text sizes
plt.rc('axes', titlesize=small_size)     # fontsize of the axes title
plt.rc('axes', labelsize=medium_size)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=small_size)    # fontsize of the tick labels
plt.rc('ytick', labelsize=small_size)    # fontsize of the tick labels
plt.rc('legend', fontsize=small_size)    # legend fontsize
plt.rc('figure', titlesize=medium_size)  # fontsize of the figure title

fig, ax = plt.subplots(figsize=(3.5, 3.5), dpi=150)
# sns.despine(f, left=True, bottom=True)
# clarity_ranking = ["I1", "SI2", "SI1", "VS2", "VS1", "VVS2", "VVS1", "IF"]

norm = plt.Normalize(star_catalog[mask].d_completeness.min(), 
                     star_catalog[mask].d_completeness.max())
sm = plt.cm.ScalarMappable(cmap=customPalette, norm=norm)
sm.set_array([])

plot_x = 'distance_s'
plot_y = 'l_sun'
plot_hue = 'd_completeness'

sns.scatterplot(x=plot_x, y=plot_y,
                hue=plot_hue,
                palette=customPalette,
                hue_norm=norm,
                #hue_order=clarity_ranking,
                sizes=(1, 8), linewidth=0,
                data=star_catalog[mask], 
                ax=ax,
                s=15)

ax.scatter(star_catalog[star_catalog.not_observed][plot_x], 
               star_catalog[star_catalog.not_observed][plot_y], 
               c=color[0], marker='v', s=15)
ax.scatter(star_catalog[star_catalog.new_observed][plot_x], 
               star_catalog[star_catalog.new_observed][plot_y], 
               c=color[1], marker='v', s=15)

# Remove the legend and add a colorbar
ax.get_legend().remove()
ax.figure.colorbar(sm, label='$\Delta$ Completeness')

legend_elements = [Line2D([0], [0], marker='v', color=color[0], label='fundamental noise only',
                          markerfacecolor=color[0], markersize=2, lw=0),
                   Line2D([0], [0], marker='v', color=color[1], label='instrumental noise only',
                          markerfacecolor=color[1], markersize=2, lw=0),
                   Line2D([0], [0], marker='o', color='k', label='in both cases',
                          markerfacecolor='k', markersize=2, lw=0),
                  ]

legend = ax.legend(handles=legend_elements, loc='upper left', title='Observed with', fontsize=small_size)

legend._legend_box.align = "left"

ax.set_yscale('log')
ax.set_xlim((0.6, 11.4))
ax.set_ylim((1.1e-4, 7e1))
ax.set_xlabel('System Distance (pc)')
ax.set_ylabel('Stellar Luminosity ($L_\odot$)')

plt.show()
fig.savefig('/home/ipa/quanz/user_accounts/fdannert/noise_paper_23/50_code/InLIFEsim/docs/source/paper_experiments/figures/sample_comparison_lay.pdf', bbox_inches='tight')

In [ ]:
significant = 0.2

star_catalog['not_observed'] = np.logical_and(star_catalog.completeness_base != 0,
                                              star_catalog.completeness_inst == 0.)
star_catalog['new_observed'] = np.logical_and(star_catalog.completeness_base == 0,
                                              star_catalog.completeness_inst != 0.)
star_catalog['sig_decrease'] = (star_catalog.d_completeness > significant)
star_catalog['sig_increase'] = (star_catalog.d_completeness < -significant)
star_catalog['same'] = np.logical_and((np.abs(star_catalog.d_completeness) < significant),
                                     star_catalog.completeness_base != 0)

In [ ]:
np.sum(star_catalog.sig_decrease)

In [ ]:
colors = ['#d7191c','#fdae61','#ffffbf','#abd9e9','#2c7bb6']
palette = sns.color_palette(colors)

sns.set_palette(palette)

small_size = 7
medium_size = 9
large_size = 10

plt.rc('font', size=small_size)          # controls default text sizes
plt.rc('axes', titlesize=small_size)     # fontsize of the axes title
plt.rc('axes', labelsize=medium_size)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=small_size)    # fontsize of the tick labels
plt.rc('ytick', labelsize=small_size)    # fontsize of the tick labels
plt.rc('legend', fontsize=small_size)    # legend fontsize
plt.rc('figure', titlesize=medium_size)  # fontsize of the figure title

f, ax = plt.subplots(figsize=(3.5, 3.5), dpi=150)

x = 'distance_s'
y = 'l_sun'

plt.scatter(star_catalog[star_catalog.new_observed][x], star_catalog[star_catalog.new_observed][y])
plt.scatter(star_catalog[star_catalog.sig_increase][x], star_catalog[star_catalog.sig_increase][y])
plt.scatter(star_catalog[star_catalog.same][x], star_catalog[star_catalog.same][y])
plt.scatter(star_catalog[star_catalog.sig_decrease][x], star_catalog[star_catalog.sig_decrease][y])
plt.scatter(star_catalog[star_catalog.not_observed][x], star_catalog[star_catalog.not_observed][y])







ax.set_yscale('log')
#ax.set_xlim((-1, 13))
#ax.set_ylim((2e-4, 3))
ax.set_xlabel('System Distance (pc)')
ax.set_ylabel('Stellar Luminosity ($L_\odot$)')

In [ ]:
star_catalog

### With paper Jens occurence statistics

In [ ]:
bus_2 = lifesim.Bus()
bus_2.build_from_config(filename='/home/ipa/quanz/user_accounts/fdannert/noise_paper_23/70_results/S01_Lay/'
                        'S01_base_bryson_hab2high.yaml')

bus_2.data.import_catalog(input_path='/home/ipa/quanz/user_accounts/fdannert/noise_paper_23/70_results/S01_Lay/'
                                   'S01_base_bryson_hab2high_catalog.hdf5',
                        noise_catalog=True)

ana = lifesim.SampleAnalysisModule(name='ana')
bus_2.add_module(ana)

ana.get_fundamental_snr()

opt_2 = lifesim.Optimizer(name='opt_2')
bus_2.add_module(opt_2)
ahgs_2 = lifesim.AhgsModule(name='ahgs_2')
bus_2.add_module(ahgs_2)

bus_2.connect(('opt_2', 'ahgs_2'))
opt_2.ahgs()

In [ ]:
star_catalog_fgk = {}
n_universe = np.unique(bus_2.data.catalog.nuniverse).shape[0]

for sn in tqdm(np.unique(bus_2.data.catalog.nstar)):
    star_ex = bus_2.data.catalog[bus_2.data.catalog.nstar == sn]
    star = star_ex.iloc[0]
    
    star_catalog_fgk[star.name_s] = {'radius_s': star.radius_s,
                                 'mass_s' : star.mass_s, 
                                 'temp_s': star.temp_s, 
                                 'distance_s': star.distance_s,
                                 'ra': star.ra, 
                                 'dec': star.dec,
                                 'stype': star.stype,
                                 'l_sun': star.l_sun,
                                }
    star_catalog_fgk[star.name_s]['no_detections_base'] = np.sum(
        bus_2.data.catalog.detected[bus_2.data.catalog.nstar == sn]
    )/n_universe
    
    star_catalog_fgk[star.name_s]['completeness_base'] = np.sum(
        bus_2.data.catalog.detected[bus_2.data.catalog.nstar == sn]
    )/bus_2.data.catalog.detected[bus_2.data.catalog.nstar == sn].shape[0]
    
    detections = []
    completeness = []
    for nu in star_ex.nuniverse:
        detections.append(np.sum(star_ex.detected[star_ex.nuniverse == nu]))
        completeness.append(np.sum(star_ex.detected[star_ex.nuniverse == nu])
                           / star_ex.detected[star_ex.nuniverse == nu].shape[0])
    detections = np.array(detections)
    completeness = np.array(completeness)
    
    '''
    fig, ax = plt.subplots(ncols=2)
    ax[0].hist(detections)
    ax[0].set_title('detections')
    ax[1].hist(completeness)
    ax[1].set_title('completeness')
    plt.show()
    '''
    
    star_catalog_fgk[star.name_s]['no_detections_base_unc'] = np.std(detections)
    star_catalog_fgk[star.name_s]['completeness_base_unc'] = np.std(completeness)
    

In [ ]:
star_catalog_fgk = pd.DataFrame(star_catalog_fgk).T

In [ ]:
f, ax = plt.subplots(figsize=(6.5, 6.5))
# sns.despine(f, left=True, bottom=True)
# clarity_ranking = ["I1", "SI2", "SI1", "VS2", "VS1", "VVS2", "VVS1", "IF"]
sns.scatterplot(x="distance_s", y="l_sun",
                hue="completeness_base",
                palette="ch:r=-.2,d=.3_r",
                #hue_order=clarity_ranking,
                sizes=(1, 8), linewidth=0,
                data=star_catalog_fgk[star_catalog_fgk.completeness_base != 0.], ax=ax)
ax.set_yscale('log')
ax.set_xlim((-1, 21))